In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import warnings

# Ignoriere alle UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Ignoriere Warnungen, die "X does not have valid feature names" enthalten
warnings.filterwarnings("ignore", message="X does not have valid feature names")

# Ignoriere Warnungen aus dem sklearn-Modul
warnings.filterwarnings("ignore", module="sklearn")

# Parameters
L = 1  # Length of the domain
T = 1.5  # Total time
nx = 20  # Number of spatial points
nt = 10000 # Number of time steps
alpha = 0.002  # Diffusion coefficient

dx = L / (nx - 1)
dt = T / nt

time = np.linspace(0, T, nt)
print(len(time))
# Stability condition
if alpha * dt / dx**2 > 0.5:
    raise ValueError("Stability condition violated")

# Initial condition: Gaussian distribution
x = np.linspace(0, L, nx)
# u = [0,0,0,0,0,0,0,0,10,20,30,20,0,0,0,0,0,0,0,0]
T = np.exp(-((x - L/2)**2) / (2 * (L/10)**2))+20

evo = []

# Time-stepping loop
for n in range(nt):
    T_new = T.copy()
    evo.append(T_new)
    for i in range(1, nx-1):
        T_new[i] = T[i] + (alpha * dt * (T[i+1] - 2*T[i] + T[i-1])) / dx**2
    T = T_new

# Parameter für den Plot

# x = np.linspace(0, L, n)  # Position entlang der Strecke
t = time        # Zeitpunkte aus der Lösung
T = evo        # Temperaturwerte
T = np.array(T).T
Initial_temp_dist = T[:, 0]
Final_temp_dist = T[:, -1]

# Erstelle die Figur und die Achse
fig = plt.figure(figsize=(10, 6))



# 1. Subplot: 3D-Plot
ax = fig.add_subplot(1, 2, 1, projection='3d')
# Erstelle ein Gitter aus Position (x) und Zeit (t)
X, T_grid = np.meshgrid(x, t)

# Transponiere die Temperaturmatrix für den Plot (da X und T_grid vertauscht sind)
Z = T.T

# Erstelle die 3D-Oberfläche
surf = ax.plot_surface(X, T_grid, Z, cmap='jet', edgecolor='none')
ax.set_title("3D-Plot der Temperaturverteilung")
ax.set_xlabel("Position x (m)")
ax.set_ylabel("Zeit t (s)")
ax.set_zlabel("Temperatur T (°C)")
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Temperatur", location="left")

# 2. Subplot: 2D-Plot (Start- und Endzustand)
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(x, Initial_temp_dist, color='green', label="Start")
ax2.plot(x, Final_temp_dist, color='red', label="Ende")
ax2.set_title("Start- und Endzustand (2D)")
ax2.set_xlabel("Position x (m)")
ax2.set_ylabel("Temperatur T (°C)")
ax2.legend()
plt.subplots_adjust(wspace=0.3)  # wspace steuert den horizontalen Abstand
# plt.subplots_adjust(hspace=0.3)
# Layout anpassen und anzeigen
# plt.tight_layout()
plt.show()

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

def preprocess_sol(train_data_normalized, train_window):
    print(train_data_normalized.shape)
    train_inout_seq= create_inout_sequences(train_data_normalized.reshape(10000, 20), train_window)
    return train_inout_seq

training_wo_num_samples = T.T

train_window = 10
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(training_wo_num_samples)
data_normalized = torch.FloatTensor(data_normalized)

train_inout_seq = preprocess_sol(data_normalized, train_window)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


# Set parameters
input_dim = 20  # Number of spatial points (nx)
hidden_dim = 64  # Number of hidden units in LSTM
output_dim = 20  # Predict temperature distribution at next step
num_layers = 2  # Number of LSTM layers
num_epochs = 20  # Number of training epochs
learning_rate = 0.001  # Learning rate
train_window = 10

# Prepare training data
train_sequences = preprocess_sol(data_normalized, train_window)
train_inputs = torch.stack([seq[0] for seq in train_sequences])
train_labels = torch.stack([seq[1] for seq in train_sequences]).squeeze(1)

# Create DataLoader for batching
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(torch.float32), labels.to(torch.float32)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Predict on the training data
model.eval()
with torch.no_grad():
    predictions = []
    inputs = train_inputs[0:1]  # Use the first sequence to predict further
    for _ in range(100):  # Generate 100 time steps
        pred = model(inputs)
        predictions.append(pred)
        inputs = torch.cat((inputs[:, 1:, :], pred.unsqueeze(1)), dim=1)

# Transform predictions back to the original scale
predictions = torch.cat(predictions).numpy()
predictions = scaler.inverse_transform(predictions)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(x, Initial_temp_dist, label="Initial Condition", linestyle="dashed")
for i in range(0, 100, 10):  # Plot every 10th prediction step
    plt.plot(x, predictions[i], label=f"Step {i+1}")
plt.xlabel("Position x (m)")
plt.ylabel("Temperature T (°C)")
plt.title("Predicted Temperature Evolution")
plt.legend()
plt.show()
